In [1]:
%load_ext autoreload
%autoreload 2
import torch
from torchvision import datasets, transforms
torch.manual_seed(42)
import json
import os
import os.path
from twosample import get_artifical_initial_clustering, utils
from twosample.affinity import fast_twosample_train_test_split, fast_twosample_train, fast_twosample_test
from twosample.architectures.resnet import resnet18
import torch.optim as optim
import numpy as np
import gc

In [2]:
hp = {}
hp['dataset'] = 'imagenet-full'
hp['initial_clustering'] = {}
hp['epochs'] = 100
hp['affinity'] = {}
hp['affinity']['batch_size'] = 128 # 64
hp['affinity']['train_ratio'] = 0.5
hp['affinity']['log_interval'] = 200
hp['device'] = torch.device('cuda')
hp['learning_rate'] = 0.001
hp['weight_decay'] = 0
hp['sobel'] = False
hp['augmentation'] = {}
hp['augmentation'] = {'random_horizontal_flip': False,
                      'random_resized_crop': False}

In [3]:
def run_experiment(hp):
    device = hp['device']
    experiment_directory = 'results/largescale/{}/'.format(hp['variant'])
    if hp['variant'] == 'artificial':
        experiment_directory += '{}_{:.2f}/'.format(hp['initial_clustering']['minimum_initial_cluster_size'],
                                                    hp['initial_clustering']['noise_ratio'])
    experiment_directory += hp['dataset']
    checkpoints_directory = experiment_directory + '/checkpoints'
    affinities_directory = experiment_directory + '/affinities'
    utils.create_directory_if_not_exists(experiment_directory)
    utils.create_directory_if_not_exists(checkpoints_directory)
    utils.create_directory_if_not_exists(affinities_directory)
    
    print('Running experiment with hyperparameters:\n{}'.format(hp))
    result = {}
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    transform_chain_train = []
    if hp['augmentation']['random_resized_crop']:
        transform_chain_train += [transforms.RandomResizedCrop(224)]
    else:
        transform_chain_train += [transforms.Resize(256), transforms.CenterCrop(224)]
    if hp['augmentation']['random_horizontal_flip']:
        transform_chain_train += [transforms.RandomHorizontalFlip()]
    transform_chain_train =  transforms.Compose(transform_chain_train + [transforms.ToTensor(), normalize])
    transform_chain_test =  transforms.Compose([transforms.Resize(256),
                                                transforms.CenterCrop(224),
                                                transforms.ToTensor(),
                                                normalize])
    dataset = datasets.ImageFolder('./datasets/' + hp['dataset'])
    
    # Check if there is already a train-test split for this dataset
    initial_clustering_filename = experiment_directory + '/initial_clustering.npy'
    if os.path.isfile(initial_clustering_filename):
        # Load previous split from file.
        print('Load initial clustering from', initial_clustering_filename)
        sample_indices_train, sample_indices_test = np.load(initial_clustering_filename).tolist()
    else:
        # If not create a new one save it to file.
        if hp['variant'] == 'artificial':
            initial_clustering = get_artifical_initial_clustering(hp, dataset)
        elif hp['variant'] == 'deepcluster10k':
            initial_clustering_raw_filename = experiment_directory + '/initial_clustering_raw.npy'
            print('Load raw initial clustering from', initial_clustering_raw_filename)
            initial_clustering = np.load(initial_clustering_raw_filename).tolist()
        sample_indices_train, sample_indices_test = fast_twosample_train_test_split(initial_clustering, hp)
        print('Save initial clustering to', initial_clustering_filename)
        np.save(initial_clustering_filename, [sample_indices_train, sample_indices_test])
    print('#Clusters:', len(sample_indices_train))

    model = resnet18(num_classes=len(sample_indices_train), sobel=hp['sobel'])
    model = model.to(device)
    optimizer = optim.Adam(filter(lambda x: x.requires_grad, model.parameters()),
                           weight_decay=hp['weight_decay'])
    loaded_epoch = 0
    
    # Check if their is a model checkpoint that can be loaded
    checkpoint_epochs = [int(checkpoint_filename.split('.')[0])
                         for checkpoint_filename in os.listdir(checkpoints_directory)]
    if len(checkpoint_epochs) > 0:
        loaded_epoch = max(checkpoint_epochs)
        checkpoint_filename = checkpoints_directory + '/{}.pth.tar'.format(loaded_epoch)
        print('Load model and optimizer state from', checkpoint_filename)
        checkpoint = torch.load(checkpoint_filename)
        model.load_state_dict(checkpoint['model'])
        optimizer.load_state_dict(checkpoint['optimizer'])
    
    optimizer.lr = hp['learning_rate']
    
    for epoch in range(loaded_epoch + 1, hp['epochs'] + 1):
        print('Epoch:', epoch)
        dataset.transform = transform_chain_train
        train_loss = fast_twosample_train(sample_indices_train, dataset, hp, model, optimizer)
        
        checkpoint_filename = checkpoints_directory + '/{}.pth.tar'.format(epoch)
        print('Save model and optimizer state to', checkpoint_filename)
        torch.save({'model': model.state_dict(),
                    'optimizer' : optimizer.state_dict()},
                   checkpoint_filename)
        
        dataset.transform = transform_chain_test
        arithmetic_means = fast_twosample_test(sample_indices_test, dataset, hp, model)
        # Set diagonal to 0 for mean calculation
        diagonal_indices = torch.arange(len(arithmetic_means), dtype=torch.long).to(device)
        arithmetic_means[diagonal_indices, diagonal_indices] = 0.
        print('Epoch: {}, Train loss: {:.2f}, mean arithmetic mean: {:.5f}'.format(epoch, train_loss, arithmetic_means.mean().item()))
        
        # Set diagonal to 1
        arithmetic_means[diagonal_indices, diagonal_indices] = 1.
        
        affinities_filename = affinities_directory + '/{}.npy'.format(epoch)
        print('Save affinities to', affinities_filename)
        np.save(affinities_filename, arithmetic_means.cpu().numpy())
        
        # Cleanup
        del arithmetic_means
        gc.collect()
        torch.cuda.empty_cache()
    
    # Cleanup
    del model
    gc.collect()
    torch.cuda.empty_cache()

In [4]:
hp['variant'] = 'artificial'
hp['epochs'] = 15
for noise_ratio in [0, 0.1, 0.3]:
    for minimum_initial_cluster_size in [50, 125, 200]:
        hp['initial_clustering']['minimum_initial_cluster_size'] = minimum_initial_cluster_size
        hp['initial_clustering']['noise_ratio'] = noise_ratio
        run_experiment(hp)

Running experiment with hyperparameters:
{'dataset': 'imagenet-full', 'initial_clustering': {'minimum_initial_cluster_size': 50, 'noise_ratio': 0}, 'epochs': 15, 'affinity': {'batch_size': 128, 'train_ratio': 0.5, 'log_interval': 200}, 'device': device(type='cuda'), 'learning_rate': 0.001, 'weight_decay': 0, 'sobel': False, 'augmentation': {'random_horizontal_flip': False, 'random_resized_crop': False}, 'variant': 'artificial'}
Load initial clustering from results/largescale/artificial/50_0.00/imagenet-full/initial_clustering.npy
#Clusters: 25577
Load model and optimizer state from results/largescale/artificial/50_0.00/imagenet-full/checkpoints/15.pth.tar
Running experiment with hyperparameters:
{'dataset': 'imagenet-full', 'initial_clustering': {'minimum_initial_cluster_size': 125, 'noise_ratio': 0}, 'epochs': 15, 'affinity': {'batch_size': 128, 'train_ratio': 0.5, 'log_interval': 200}, 'device': device(type='cuda'), 'learning_rate': 0.001, 'weight_decay': 0, 'sobel': False, 'augmenta

/home/chris/miniconda2/envs/pytorch4/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1835008 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))


Batches processed: 800/4986 (16.0%), ETA: 52m:10s, Loss: 0.422565
Batches processed: 1000/4986 (20.1%), ETA: 50m:51s, Loss: 0.430738
Batches processed: 1200/4986 (24.1%), ETA: 48m:22s, Loss: 0.442343
Batches processed: 1400/4986 (28.1%), ETA: 46m:08s, Loss: 0.465960


/home/chris/miniconda2/envs/pytorch4/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))


Batches processed: 1600/4986 (32.1%), ETA: 43m:35s, Loss: 0.473897
Batches processed: 1800/4986 (36.1%), ETA: 41m:23s, Loss: 0.522161
Batches processed: 2000/4986 (40.1%), ETA: 38m:51s, Loss: 0.455657
Batches processed: 2200/4986 (44.1%), ETA: 36m:14s, Loss: 0.436924
Batches processed: 2400/4986 (48.1%), ETA: 33m:41s, Loss: 0.448150
Batches processed: 2600/4986 (52.1%), ETA: 31m:15s, Loss: 0.446584
Batches processed: 2800/4986 (56.2%), ETA: 28m:36s, Loss: 0.456083
Batches processed: 3000/4986 (60.2%), ETA: 26m:03s, Loss: 0.482857
Batches processed: 3200/4986 (64.2%), ETA: 23m:24s, Loss: 0.446037
Batches processed: 3400/4986 (68.2%), ETA: 20m:50s, Loss: 0.495553
Batches processed: 3600/4986 (72.2%), ETA: 18m:14s, Loss: 0.535981
Batches processed: 3800/4986 (76.2%), ETA: 15m:38s, Loss: 0.485873
Batches processed: 4000/4986 (80.2%), ETA: 13m:02s, Loss: 0.443628
Batches processed: 4200/4986 (84.2%), ETA: 10m:23s, Loss: 0.421817
Batches processed: 4400/4986 (88.2%), ETA: 07m:44s, Loss: 0.42

...

Batches processed: 1800/5017 (35.9%), ETA: 38m:20s
Batches processed: 2000/5017 (39.9%), ETA: 35m:56s
Batches processed: 2200/5017 (43.9%), ETA: 33m:32s


/home/chris/miniconda2/envs/pytorch4/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 19660800 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/chris/miniconda2/envs/pytorch4/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 18481152 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/chris/miniconda2/envs/pytorch4/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 37093376 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/chris/miniconda2/envs/pytorch4/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 39976960 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/chris/mini

Batches processed: 2400/5017 (47.8%), ETA: 31m:07s
Batches processed: 2600/5017 (51.8%), ETA: 28m:38s
Batches processed: 2800/5017 (55.8%), ETA: 26m:16s
Batches processed: 3000/5017 (59.8%), ETA: 23m:54s
Batches processed: 3200/5017 (63.8%), ETA: 21m:28s
Batches processed: 3400/5017 (67.8%), ETA: 19m:06s
Batches processed: 3600/5017 (71.8%), ETA: 16m:44s
Batches processed: 3800/5017 (75.7%), ETA: 14m:22s
Batches processed: 4000/5017 (79.7%), ETA: 11m:59s
Batches processed: 4200/5017 (83.7%), ETA: 09m:36s
Batches processed: 4400/5017 (87.7%), ETA: 07m:15s
Batches processed: 4600/5017 (91.7%), ETA: 04m:53s


/home/chris/miniconda2/envs/pytorch4/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1835008 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))


Batches processed: 4800/5017 (95.7%), ETA: 02m:33s
Batches processed: 5000/5017 (99.7%), ETA: 12s
Batches processed: 5017/5017 (100.0%), ETA: 0s
Epoch: 15, Train loss: 294.40, mean arithmetic mean: 0.78089
Save affinities to results/largescale/artificial/200_0.30/imagenet-full/affinities/15.npy
